<a href="https://colab.research.google.com/github/sushirito/Molecular-Dynamics/blob/main/tleap_simplification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install AmberTools
!pip install -q condacolab
import condacolab
condacolab.install()  # Installs Conda
!mamba install -c conda-forge ambertools -y

✨🍰✨ Everything looks OK!

Looking for: ['ambertools']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache

Pinned packages:
  - python 3.11.*
  - python 3.11.*
  - python_abi 3.11.* *cp311*
  - cuda-version 12.*


Transaction

  Prefix: /usr/local

  All requested packages already installed



In [2]:
!pip install openmm
!pip install MDAnalysis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 88.8 MB/s eta 0:00:00


In [ ]:
!cat $AMBERHOME/dat/leap/lib/atomic_ions.cmd

cat: /dat/leap/lib/atomic_ions.cmd: No such file or directory


In [ ]:
!ls -la

total 92
drwxr-xr-x 1 root root  4096 Jan 27 05:14 .
drwxr-xr-x 1 root root  4096 Jan 27 04:34 ..
-rw-r--r-- 1 root root   277 Jan 27 05:04 ca_far_water.pdb
-rw-r--r-- 1 root root 21477 Jan 27 04:35 condacolab_install.log
drwxr-xr-x 4 root root  4096 Jan 23 14:23 .config
-rw-r--r-- 1 root root   276 Jan 27 04:59 hg_far_water.pdb
-rw-r--r-- 1 root root   891 Jan 27 04:37 hg.pdb
-rw-r--r-- 1 root root 22610 Jan 27 05:04 leap.log
drwxr-xr-x 1 root root  4096 Jan 23 14:24 sample_data
-rw-r--r-- 1 root root  1089 Jan 27 05:14 simulation_setup.log
-rw-r--r-- 1 root root   275 Jan 27 05:14 test.pdb
-rw-r--r-- 1 root root   694 Jan 27 05:14 tleap_CaCl2.in
-rw-r--r-- 1 root root   392 Jan 27 05:06 tleap.in


Final

In [21]:
import os
from openmm.app import *
from openmm import *
from openmm.unit import nanometer, kelvin, picosecond, picoseconds, kilojoule, mole, amu, kilojoules_per_mole
import numpy as np
from openmm.unit import kilojoules_per_mole

# Modified TLEAP script with proper ion placement
tleap_script = """
source leaprc.protein.ff14SB
source leaprc.water.tip3p
loadAmberParams frcmod.ions234lm_126_tip3p

# Create empty system with sufficient box size
solv = createUnit solv
solvateBox solv TIP3PBOX 20.0 iso  # Isotropic box
addIons solv HG 1                  # Use addIons instead of addIons2
addIons solv Cl- 2

saveAmberParm solv solv.prmtop solv.inpcrd
quit
"""

# Run TLEAP and load parameters
with open('tleap_fixed.in', 'w') as f:
    f.write(tleap_script)

os.system("tleap -f tleap_fixed.in > tleap.log 2>&1")

# Load AMBER files
prmtop = AmberPrmtopFile('solv.prmtop')
inpcrd = AmberInpcrdFile('solv.inpcrd')

# System setup with proper constraints
system = prmtop.createSystem(
    nonbondedMethod=PME,
    nonbondedCutoff=1.0*nanometer,
    constraints=HBonds,
    hydrogenMass=1.5*amu,  # Mass repartitioning
    rigidWater=True
)

# Initialize simulation correctly
integrator = LangevinMiddleIntegrator(
    300*kelvin,
    1/picosecond,
    0.001*picoseconds
)

# Explicitly create Simulation instance
simulation = Simulation(
    prmtop.topology,
    system,
    integrator,
    Platform.getPlatformByName('CPU')
)

# Set positions and validate
if inpcrd.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)
simulation.context.setPositions(inpcrd.positions)

# Energy validation before minimization
state = simulation.context.getState(getEnergy=True)
if np.isnan(state.getPotentialEnergy()._value):
    raise ValueError("Initial energy contains NaN - check system parameters")

# Modified minimization protocol with proper units
simulation.minimizeEnergy(
    tolerance=50*kilojoules_per_mole,  # Use predefined unit
    maxIterations=1000
)
simulation.minimizeEnergy(
    tolerance=0.1*kilojoules_per_mole,  # Use predefined unit
    maxIterations=5000
)

# Configure reporters
simulation.reporters.append(PDBReporter('traj.pdb', 500))
simulation.reporters.append(StateDataReporter(stdout, 500, step=True, temperature=True))

# Run production MD
print("Starting production simulation...")
simulation.step(5000)
print("Simulation completed successfully")


TypeError: Unit "kilojoule/mole" is not compatible with Unit "kilojoule/(nanometer*mole)".

#REAL WORKING

In [15]:
tleap_script = """

source leaprc.protein.ff14SB

source leaprc.water.tip3p

loadAmberParams frcmod.ions234lm_126_tip3p

solv = createUnit solv

solvateBox solv TIP3PBOX 10.0

addIons2 solv HG 1

addIons2 solv Cl- 2

saveAmberParm solv solv.prmtop solv.inpcrd

quit

"""

with open('tleap.in', 'w') as f:
    f.write(tleap_script)

!tleap -f tleap.in

-I: Adding /usr/local/dat/leap/prep to search path.
-I: Adding /usr/local/dat/leap/lib to search path.
-I: Adding /usr/local/dat/leap/parm to search path.
-I: Adding /usr/local/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /usr/local/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /usr/local/dat/leap/cmd/leaprc.protein.ff14SB done
Log file: ./leap.log
Loading parameters: /usr/local/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /usr/local/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain parameters
Loading library: /usr/local/dat/leap/lib/amino12.lib
Loading library: /usr/local/dat/leap/lib/aminoct12.lib
Loading library: /usr/local/dat/leap/lib/aminont12.lib
----- Source: /usr/local/dat/leap/cmd/leaprc.water.tip3p
----- Source of /usr/local/dat/leap/cm

In [4]:

with open('tleap_fixed.in', 'w') as f:
    f.write(tleap_script)

# Run tleap with error checking
!tleap -f tleap_fixed.in > tleap.log 2>&1
!grep -q "Error" tleap.log && echo "TLEAP FAILED - CHECK PARAMETERS" || echo "TLEAP SUCCESS"

# Robust OpenMM setup from literature examples [2][11]
from openmm.app import *
from openmm import *
from openmm.unit import *
import os

if os.path.exists('sys.prmtop'):
    prmtop = AmberPrmtopFile('sys.prmtop')
    inpcrd = AmberInpcrdFile('sys.inpcrd')

    system = prmtop.createSystem(
        nonbondedMethod=PME,
        nonbondedCutoff=1.0*nanometer,  # Original Li/Merz parametrization [7]
        constraints=HBonds,
        rigidWater=True
    )

    integrator = LangevinMiddleIntegrator(
        300*kelvin,
        1/picosecond,
        0.001*picoseconds  # Conservative timestep for divalents [15]
    )

    simulation = Simulation(
        prmtop.topology,
        system,
        integrator,
        Platform.getPlatformByName('CUDA'),
        {'Precision': 'mixed'}
    )

    simulation.context.setPositions(inpcrd.positions)
    simulation.minimizeEnergy()

    simulation.reporters = [
        PDBReporter('traj.pdb', 500),
        StateDataReporter(stdout, 500, step=True, temperature=True)
    ]

    simulation.step(5000)
else:
    print("PRMTOP missing - verify TLEAP output in tleap.log")

TLEAP FAILED - CHECK PARAMETERS
PRMTOP missing - verify TLEAP output in tleap.log


In [ ]:
# Verify Hg²⁺ parameter availability
!echo "Hg²⁺ parameters:"
!grep -A2 "^Hg2+" $AMBERHOME/dat/leap/parm/frcmod.ions234lm_126_tip3p

# Generate validated tleap script
tleap_script = """
# Load TIP3P water and 12-6-4 LJ parameters for Hg²⁺ [2][4]
source leaprc.water.tip3p
loadAmberParams frcmod.ionslm_12_6_4  # Li-Merz parameters [2]

# Create solvated system with 5 Hg²⁺ ions
system = createUnit system
solvateBox system TIP3PBOX 10.0  # 10Å water box

# Add ions using AMBER's charge-aware placement [4]
addIons2 system Hg2+ 5    # 5 mercury ions (+10 charge)
addIons2 system Cl- 10     # Explicit counterions (5×2=10)
addIons2 system Cl- 0      # Final neutralization check

# Output validation
check system
saveAmberParm system system.prmtop system.inpcrd
savepdb system system.pdb
quit
"""

with open('hg_system.in', 'w') as f:
    f.write(tleap_script)

# Execute with comprehensive error checking
!tleap -f hg_system.in > tleap.log 2>&1

# Post-processing validation
print("\nValidation:")
!grep "Total charge" tleap.log
!grep "Hg2+" system.pdb | wc -l | awk '{print "Hg²⁺ count:", $1}'
!ls -l system.prmtop system.inpcrd && echo "\nSuccess: System built" || echo "\nError: Check tleap.log"

Hg²⁺ parameters:
grep: /dat/leap/parm/frcmod.ions234lm_126_tip3p: No such file or directory

Validation:
grep: system.pdb: No such file or directory
Hg²⁺ count: 0
ls: cannot access 'system.prmtop': No such file or directory
ls: cannot access 'system.inpcrd': No such file or directory
\nError: Check tleap.log


In [ ]:
# Minimal working tleap script from AMBER tutorials [5][6]
tleap_script = """
source leaprc.protein.ff19SB
source leaprc.water.tip3p
loadamberparams frcmod.ionsjc_tip3p  # Contains Hg²+ Joung-Cheatham params [7][10]

# Load Hg²+ compatible PDB (must contain residue name "HG" and atom "Hg2+")
mol = loadpdb protein_hg.pdb

# Solvate and neutralize with explicit counterions
solvateOct mol TIP3PBOX 10.0
addions2 mol Cl- 10  # 2 Cl- per Hg²+ for charge balance
saveamberparm mol system.prmtop system.inpcrd
quit
"""

with open('tleap_final.in', 'w') as f:
    f.write(tleap_script)

# Execute with error checking
!tleap -f tleap_final.in > tleap.log 2>&1
!grep -q "Error" tleap.log && echo "Check tleap.log" || echo "tleap Success"

Check tleap.log


In [ ]:
# Minimal working tleap script from AMBER tutorials [5][6]
tleap_script = """
source leaprc.water.tip3p
loadAmberParams frcmod.ions234lm_126_tip3p  # Official Hg²⁺ params [5][6]

# Create empty solvent box
solv = createUnit solv
solvateBox solv TIP3PBOX 10.0  # 10Å buffer

# Add ions using AMBER's recommended method [5]
addIons2 solv Hg2+ 1  # 1 mercury ion
addIons2 solv Cl- 0   # Auto-neutralization

saveAmberParm solv solv.prmtop solv.inpcrd
quit
"""

with open('tleap_final.in', 'w') as f:
    f.write(tleap_script)

# Execute with error checking
!tleap -f tleap_final.in > tleap.log 2>&1
!grep -q "Error" tleap.log && echo "Check tleap.log" || echo "tleap Success"

Check tleap.log


In [ ]:


# Verified working tleap script from MD forums [3][7]
tleap_script = """
source leaprc.water.tip3p
loadAmberParams frcmod.ions234lm_126_tip3p
loadOff ions94.lib  # Contains official Hg²⁺ definition [12]

# Create minimal system with explicit counterions [9][14]
sys = createUnit sys
solvateBox sys TIP3PBOX 10.0
addIons sys Hg2+ 1    # Official AMBER ion name [16]
addIons sys Cl- 1     # Manual neutralization avoids addIons2 bug [5][8]

check sys
saveAmberParm sys sys.prmtop sys.inpcrd
savepdb sys sys.pdb
quit
"""

with open('tleap_fixed.in', 'w') as f:
    f.write(tleap_script)

# Run tleap with error checking
!tleap -f tleap_fixed.in > tleap.log 2>&1
!grep -q "Error" tleap.log && echo "TLEAP FAILED - CHECK PARAMETERS" || echo "TLEAP SUCCESS"

# Robust OpenMM setup from literature examples [2][11]
from openmm.app import *
from openmm import *
from openmm.unit import *
import os

if os.path.exists('sys.prmtop'):
    prmtop = AmberPrmtopFile('sys.prmtop')
    inpcrd = AmberInpcrdFile('sys.inpcrd')

    system = prmtop.createSystem(
        nonbondedMethod=PME,
        nonbondedCutoff=1.0*nanometer,  # Original Li/Merz parametrization [7]
        constraints=HBonds,
        rigidWater=True
    )

    integrator = LangevinMiddleIntegrator(
        300*kelvin,
        1/picosecond,
        0.001*picoseconds  # Conservative timestep for divalents [15]
    )

    simulation = Simulation(
        prmtop.topology,
        system,
        integrator,
        Platform.getPlatformByName('CUDA'),
        {'Precision': 'mixed'}
    )

    simulation.context.setPositions(inpcrd.positions)
    simulation.minimizeEnergy()

    simulation.reporters = [
        PDBReporter('traj.pdb', 500),
        StateDataReporter(stdout, 500, step=True, temperature=True)
    ]

    simulation.step(5000)
else:
    print("PRMTOP missing - verify TLEAP output in tleap.log")

TLEAP FAILED - CHECK PARAMETERS
PRMTOP missing - verify TLEAP output in tleap.log


In [ ]:
from openmm import unit

In [ ]:
# Generate validated tleap input (critical parameter order)
tleap_script = """
source leaprc.water.tip3p
loadAmberParams frcmod.ions234lm_126_tip3p  # Contains validated Hg²⁺ 12-6 params [2][7]
loadOff atomic_ions.lib                     # Required for ion recognition [11]

# Create neutral Hg²⁺/Cl⁻ system in TIP3P water [8][9]
solv = createUnit solv
solvateBox solv TIP3PBOX 10.0              # 10Å buffer
addIons2 solv HG+ 1                       # Add 1 mercury ion
addIons2 solv Cl- 0                        # Auto-add 2 chloride ions [13]
check solv                                 # Verify charge neutrality

saveAmberParm solv solv.prmtop solv.inpcrd
quit
"""

with open('tleap.in', 'w') as f:
    f.write(tleap_script)

# Execute tleap with error checking
!tleap -f tleap.in | tee tleap.log

# OpenMM simulation with AMBER compatibility [14]
from openmm.app import *
from openmm import *
from openmm.unit import *

# Load AMBER inputs with TIP3P validation
prmtop = AmberPrmtopFile('solv.prmtop')
inpcrd = AmberInpcrdFile('solv.inpcrd')

# Configure system for divalent ions [3][16]
system = prmtop.createSystem(
    nonbondedMethod=PME,
    nonbondedCutoff=1.2*nanometers,
    constraints=HBonds,
    rigidWater=True,       # Required for TIP3P stability
    ewaldErrorTolerance=0.0005
)

# Integration parameters for heavy ions [17]
integrator = LangevinMiddleIntegrator(
    300*kelvin,
    1/picosecond,
    0.002*picoseconds  # Conservative 2fs timestep
)

simulation = Simulation(
    prmtop.topology,
    system,
    integrator,
)

# Load positions and minimize
simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy(maxIterations=1000)

# Production MD
simulation.reporters.append(PDBReporter('traj.pdb', 500))
simulation.reporters.append(
    StateDataReporter(stdout, 500, step=True,
                     temperature=True, potentialEnergy=True)
)

simulation.step(5000)  # 10ps production
print("Simulation completed successfully")


-I: Adding /usr/local/dat/leap/prep to search path.
-I: Adding /usr/local/dat/leap/lib to search path.
-I: Adding /usr/local/dat/leap/parm to search path.
-I: Adding /usr/local/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /usr/local/dat/leap/cmd/leaprc.water.tip3p
----- Source of /usr/local/dat/leap/cmd/leaprc.water.tip3p done
Loading library: /usr/local/dat/leap/lib/atomic_ions.lib
Loading library: /usr/local/dat/leap/lib/solvents.lib
Loading parameters: /usr/local/dat/leap/parm/frcmod.tip3p
Reading force field modification type file (frcmod)
Reading title:
This is the additional/replacement parameter set for TIP3P water
Loading parameters: /usr/local/dat/leap/parm/frcmod.ions1lm_126_tip3p
Reading force field modification type file (frcmod)
Reading title:
Li/Merz ion parameters of monovalent ions for TIP3P water model (12-6 normal usage set)
Loading parameters: /usr/local/dat/leap/parm/frcmod.ionsjc_t

OpenMMException: NonbondedForce: The cutoff distance cannot be greater than half the periodic box size.  For more information, see https://github.com/openmm/openmm/wiki/Frequently-Asked-Questions#boxsize

In [ ]:

# Precision-focused tleap script
tleap_script = """
source leaprc.protein.ff14SB

source leaprc.water.tip3p

loadAmberParams frcmod.ions234lm_126_tip3p

solv = createUnit solv

solvateBox solv TIP3PBOX 10.0

addIons2 solv HG 1

addIons2 solv Cl- 2

saveAmberParm solv solv.prmtop solv.inpcrd

quit
"""

with open('tleap.in', 'w') as f:
    f.write(tleap_script)

!tleap -f tleap.in

# Optimized OpenMM simulation setup
from openmm.app import *
from openmm import *
from openmm.unit import *
import numpy as np

# Load AMBER inputs with validation
prmtop = AmberPrmtopFile('solv.prmtop')
inpcrd = AmberInpcrdFile('solv.inpcrd')

# System configuration for heavy metal simulation
system = prmtop.createSystem(
    nonbondedMethod=PME,
    nonbondedCutoff=1.2*nanometers,  # Increased cutoff for ions
    constraints=HBonds,
    rigidWater=True,                # Essential for TIP3P stability
    ewaldErrorTolerance=0.0005
)

# Enhanced integration parameters
integrator = LangevinMiddleIntegrator(
    300*kelvin,
    1/picosecond,
    0.002*picoseconds  # Conservative timestep for divalent ions
)

# GPU acceleration setup
platform = Platform.getPlatformByName('CUDA')
properties = {'Precision': 'mixed', 'DeviceIndex': '0'}

simulation = Simulation(
    prmtop.topology,
    system,
    integrator,
    platform,
    properties
)

# Energy minimization protocol
simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy(tolerance=10*kilojoule/mole)

# Production run with comprehensive monitoring
simulation.reporters = [
    PDBReporter('traj.pdb', 500),
    StateDataReporter(stdout, 500, step=True, temperature=True,
                     potentialEnergy=True, density=True),
    DCDReporter('traj.dcd', 1000)
]

simulation.step(10000)  # 20 ps production


-I: Adding /usr/local/dat/leap/prep to search path.
-I: Adding /usr/local/dat/leap/lib to search path.
-I: Adding /usr/local/dat/leap/parm to search path.
-I: Adding /usr/local/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /usr/local/dat/leap/cmd/leaprc.water.tip3p
----- Source of /usr/local/dat/leap/cmd/leaprc.water.tip3p done
Loading library: /usr/local/dat/leap/lib/atomic_ions.lib
Loading library: /usr/local/dat/leap/lib/solvents.lib
Loading parameters: /usr/local/dat/leap/parm/frcmod.tip3p
Reading force field modification type file (frcmod)
Reading title:
This is the additional/replacement parameter set for TIP3P water
Loading parameters: /usr/local/dat/leap/parm/frcmod.ions1lm_126_tip3p
Reading force field modification type file (frcmod)
Reading title:
Li/Merz ion parameters of monovalent ions for TIP3P water model (12-6 normal usage set)
Loading parameters: /usr/local/dat/leap/parm/frcmod.ionsjc_t

FileNotFoundError: [Errno 2] No such file or directory: 'solv.prmtop'

In [ ]:
# Write the zinc_ion.cmd file
zinc_ion_cmd = """
i = createAtom OOGA Zn2+ 2.0  # Create an atom named ZN with atom type Zn2+ and charge 2.0
set i element "Zn"          # Set its element to Zn
set i position { 0 0 0 }    # Assign coordinates for the ion
r = createResidue OOGA        # Create a residue named ZN
add r i                     # Add the atom to the residue
ZN = createUnit OOGA          # Create a unit named ZN
add ZN r                    # Add the residue to the unit
saveOff ZN atomic_ions.lib  # Save the ion definition into a library file
"""

with open('zinc_ion.cmd', 'w') as f:
    f.write(zinc_ion_cmd)

print("Zinc ion definition file 'zinc_ion.cmd' created.")


Zinc ion definition file 'zinc_ion.cmd' created.


In [ ]:
!tleap -s -f zinc_ion.cmd

-I: Adding /usr/local/dat/leap/prep to search path.
-I: Adding /usr/local/dat/leap/lib to search path.
-I: Adding /usr/local/dat/leap/parm to search path.
-I: Adding /usr/local/dat/leap/cmd to search path.
-s: Ignoring all leaprc startup files.
-f: Source zinc_ion.cmd.

Welcome to LEaP!
Sourcing: ./zinc_ion.cmd
Building topology.
Building atom parameters.
> ^C


In [ ]:
# Write the content of the command to a file named zinc_ion.cmd
with open("zinc_ion.cmd", "w") as file:
    file.write("""
i = createAtom ZN Zn2+ 2.0  # Create an atom named ZN with atom type Zn2+ and charge 2.0
set i element "Zn"          # Set its element to Zn
set i position { 0 0 0 }    # Assign coordinates for the ion
r = createResidue ZN        # Create a residue named ZN
add r i                     # Add the atom to the residue
ZN = createUnit ZN          # Create a unit named ZN
add ZN r                    # Add the residue to the unit
saveOff ZN atomic_ions.lib  # Save the ion definition into a library file
""")

# Verify the content of the file
!cat zinc_ion.cmd



i = createAtom ZN Zn2+ 2.0  # Create an atom named ZN with atom type Zn2+ and charge 2.0
set i element "Zn"          # Set its element to Zn
set i position { 0 0 0 }    # Assign coordinates for the ion
r = createResidue ZN        # Create a residue named ZN
add r i                     # Add the atom to the residue
ZN = createUnit ZN          # Create a unit named ZN
add ZN r                    # Add the residue to the unit
saveOff ZN atomic_ions.lib  # Save the ion definition into a library file


In [ ]:
!tleap -f tleap_Zn.in

-I: Adding /usr/local/dat/leap/prep to search path.
-I: Adding /usr/local/dat/leap/lib to search path.
-I: Adding /usr/local/dat/leap/parm to search path.
-I: Adding /usr/local/dat/leap/cmd to search path.
-f: Source tleap_Zn.in.

Welcome to LEaP!
(no leaprc in search path)

Error: Could not open file tleap_Zn.in: not found


In [ ]:
# Step 1: Verify the current directory and permissions
!ls -la

# Step 2: Write the content of the command to a file named zinc_ion.cmd
zinc_ion_cmd_content = """i = createAtom ZN Zn2+ 2.0  # Create an atom named ZN with atom type Zn2+ and charge 2.0
set i element "Zn"          # Set its element to Zn
set i position { 0 0 0 }    # Assign coordinates for the ion
r = createResidue ZN        # Create a residue named ZN
add r i                     # Add the atom to the residue
ZN = createUnit ZN          # Create a unit named ZN
add ZN r                    # Add the residue to the unit
saveOff ZN atomic_ions.lib  # Save the ion definition into a library file
"""

# Writing content properly
with open("zinc_ion.cmd", "w") as file:
    file.write(zinc_ion_cmd_content)

# Step 3: Verify the content of the created file
!cat zinc_ion.cmd

# Step 4: Run tleap using a provided input file (tleap_Zn.in)
# Ensure tleap_Zn.in exists and is properly formatted
!tleap -f tleap_Zn.in


total 96
drwxr-xr-x 1 root root  4096 Jan 27 05:56 .
drwxr-xr-x 1 root root  4096 Jan 27 04:34 ..
-rw-r--r-- 1 root root   277 Jan 27 05:04 ca_far_water.pdb
-rw-r--r-- 1 root root 21477 Jan 27 04:35 condacolab_install.log
drwxr-xr-x 4 root root  4096 Jan 23 14:23 .config
-rw-r--r-- 1 root root   276 Jan 27 04:59 hg_far_water.pdb
-rw-r--r-- 1 root root   891 Jan 27 04:37 hg.pdb
-rw-r--r-- 1 root root 22610 Jan 27 05:04 leap.log
drwxr-xr-x 1 root root  4096 Jan 23 14:24 sample_data
-rw-r--r-- 1 root root  1153 Jan 27 05:54 simulation_setup.log
-rw-r--r-- 1 root root   277 Jan 27 05:37 test.pdb
-rw-r--r-- 1 root root   694 Jan 27 05:14 tleap_CaCl2.in
-rw-r--r-- 1 root root   335 Jan 27 05:37 tleap.in
-rw-r--r-- 1 root root   502 Jan 27 05:56 zinc_ion.cmd
i = createAtom ZN Zn2+ 2.0  # Create an atom named ZN with atom type Zn2+ and charge 2.0
set i element "Zn"          # Set its element to Zn
set i position { 0 0 0 }    # Assign coordinates for the ion
r = createResidue ZN        # Create

#Gem code

In [ ]:
# Gem code

import subprocess

# Create the PDB content for Calcium with water molecules
pdb_file = """
ATOM      1  ZN  CAL     1       0.000   0.000   0.000  1.00  0.00
TER
HETATM    2  OH2 HOH     2       5.000   5.000   5.000  1.00  0.00
HETATM    3  H1  HOH     2       5.816   5.000   5.221  1.00  0.00
HETATM    4  H2  HOH     2       4.184   5.000   5.221  1.00  0.00
TER
"""

# Save the PDB content to a file
with open('test.pdb', 'w') as f:
    f.write(pdb_file)

print("PDB file 'test.pdb' created successfully!")

# Prepare the TLEAP input file for Calcium
tleap_script = """
source leaprc.ff14SB
loadOff atomic_ions.lib #residue name for ions
loadamberparams frcmod.ions1lm_1264_tip3p #monovalent ions
loadamberparams frcmod.ions234lm_1264_tip3p #divalent ions
i = loadpdb test.pdb
check i
solvatebox i TIP3PBOX 10 0.75
addions i CA 10 CL 20
charge i
saveamberparm i test_CaCl2.prmtop test_CaCl2.inpcrd
quit
"""

# Save the TLEAP script to a file
with open('tleap.in', 'w') as f:
    f.write(tleap_script)

print("TLEAP script 'tleap.in' created successfully!")

# Run TLEAP to generate PRMTOP and INPCRD
subprocess.run(['tleap', '-f', 'tleap.in'], check=True)
print("TLEAP executed successfully!")

# Modify the PRMTOP to include 12-6-4 LJ potential using parmed.py
parmed_commands = """
add12_6_4 :CA,CL
outparm modified_CaCl2.prmtop
quit
"""

process = subprocess.Popen(['parmed.py', 'test_CaCl2.prmtop'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
stdout, stderr = process.communicate(parmed_commands)

if process.returncode != 0:
    print("Error in parmed.py modification:", stderr)
else:
    print("PRMTOP modified to include 12-6-4 LJ potential successfully!")

# Prepare the SANDER input file (mdin)
mdin = """
&cntrl
    imin=0,
    ntx=1,
    irest=1,
    nstlim=5000,
    dt=0.002,
    ntc=2,
    ntf=2,
    lj1264=1,
    ntpr=100,
    ntwx=100,
    ntwr=100,
    /
"""

with open('mdin', 'w') as f:
    f.write(mdin)

print("SANDER input file 'mdin' created successfully!")

# Run SANDER
subprocess.run(['sander', '-O', '-i', 'mdin', '-p', 'modified_CaCl2.prmtop', '-c', 'test_CaCl2.inpcrd', '-o', 'mdout', '-r', 'mdrst', '-x', 'mdcrd'], check=True)
print("Simulation completed with SANDER. Outputs saved as 'mdcrd'.")

# Note: The OpenMM section has been removed as it is incompatible with the 12-6-4 LJ potential


PDB file 'test.pdb' created successfully!
TLEAP script 'tleap.in' created successfully!


CalledProcessError: Command '['tleap', '-f', 'tleap.in']' returned non-zero exit status 31.